In [ ]:
import numpy as np
import pandas as pd
from lmfit import Model
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import special
import scipy
import math
import scipy.integrate as integrate
import scipy.special as special
import matplotlib.patches as mpatches
import runipy
from runipy.notebook_runner import NotebookRunner
#from IPython.nbformat.current import read
from nbformat.current import read
import runpy
import math

In [ ]:
#names of colums from data
names=['z [mm]' ,'sigma2_x [mm]','sigma2_y [mm]', 'NaN']
namesYi=['Component', 'drift length', 'system length [m]','sigma2_x [cm]' ,'sigma2_y [cm]' ]

In [ ]:
#reading the data
#file = r"C:\Users\Stefan\Documents\Masterarbeit\G4Beamline\BL1U\Many_VD\Results_II.txt"
file = r"C:\Users\Stefan\Documents\Masterarbeit\G4Beamline\BL1U\Many_VD\Results_IIII.txt"
Yi= r"C:\Users\Stefan\Documents\Masterarbeit\Yi-Nong\1U_OM\1U_OM\Beamsize.txt"

In [ ]:
#Data
DataFrame = pd.read_csv(file, header=None, sep=' ', comment='#', names=names)
DataFrame_dtype = 'float'
DataFrameYi = pd.read_csv(Yi, header=None, sep='\t', comment='#', names=namesYi)
DataFrameYi_dtype = 'float'
DataFrame

In [ ]:
# plot of beamsize x in BL1U 
plt.figure(1) 
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots()
L1, = plt.plot(DataFrame['z [mm]']/1000,abs(DataFrame['sigma2_x [mm]']),'g-',linewidth=3, label="2\u03C3_x G4B") 
L2, = plt.plot(DataFrame['z [mm]']/1000,-abs(DataFrame['sigma2_y [mm]']),'g--',linewidth=3, label="2\u03C3_y G4B")
L3, = plt.plot(DataFrameYi['system length [m]'],DataFrameYi['sigma2_x [cm]']*10,'r-',linewidth=3, label="2\u03C3_x TROPTR") 
L4, = plt.plot(DataFrameYi['system length [m]'],-DataFrameYi['sigma2_y [cm]']*10,'r--',linewidth=3, label="2\u03C3_y TROPTR") 
ax.axhline(y=0, color='k')
plt.legend(handles=[L1,L2,L3,L4], fontsize=13)  #,L3,L4
plt.xticks(np.linspace(0, 30, 7,endpoint=True)) # (start, end, points [+1])
plt.yticks(np.linspace(-25, 25, 11,endpoint=True))
plt.text(0.02,-22, 'HARP0', fontsize=12, rotation=90, color='b')
plt.text(19.96,-22, 'HARP1', fontsize=12, rotation=90, color='b')
plt.text(28.341,-22, 'HARP2', fontsize=12, rotation=90, color='b')
plt.text(31.11,-22, 'HARP3', fontsize=12, rotation=90, color='b')
plt.axvline(0.01, ymax=0.55, linewidth=3, color = 'grey', alpha=0.3)
plt.axvline(19.910, ymax=0.55, linewidth=3, color = 'grey', alpha=0.3)
plt.axvline(28.281, ymax=0.75, linewidth=3, color = 'grey', alpha=0.3)
plt.axvline(31.1,  linewidth=3, color = 'grey', alpha=0.3)

plt.text(0.52952,-22, 'Q1', fontsize=12, rotation=90, color='b')
plt.text(1.2747,-22, 'Q2', fontsize=12, rotation=90, color='b')
plt.text(2.55,-22, 'Q3', fontsize=12, rotation=90, color='b')
plt.text(4.75,-22, '1VB1', fontsize=12, rotation=90, color='b')
plt.text(6.9,-22, 'Q4', fontsize=12, rotation=90, color='b')
plt.text(7.65,-22, 'Q5', fontsize=12, rotation=90, color='b')
plt.text(8.3897,-22, 'Q6', fontsize=12, rotation=90, color='b')
plt.text(10.03,-22, 'KICKER', fontsize=12, rotation=90, color='b')
#plt.text(13.471,-22, '1BVB2', fontsize=12, rotation=90, color='b')
plt.text(15.9,-22, 'SEPTUM', fontsize=12, rotation=90, color='b')
#plt.text(19.3,-22, 'CBY0', fontsize=12, rotation=90, color='b')
plt.text(21.45,-22, '1UB0', fontsize=12, rotation=90, color='b')
plt.text(26.47,-22, '1UQ1', fontsize=12, rotation=90, color='b')
plt.text(27.05,-22, '1UQ2', fontsize=12, rotation=90, color='b')
plt.text(29.61,-22, 'COLL', fontsize=12, rotation=90, color='b')
#plt.text(30.5,-22, 'BEW', fontsize=12, rotation=90, color='b')
plt.text(31.8,-22, 'TARGET', fontsize=12, rotation=90, color='b')
#Positions of Componets in old G4Beamline model
#plt.text(1.41732,-27, 'Q1', fontsize=9, rotation=90, color='r')
#plt.text(2.16408,-27, 'Q2', fontsize=9, rotation=90, color='r')
#plt.text(3.48488,-27, 'Q3', fontsize=9, rotation=90, color='r')
##Regarding Beamline 1U Geometry, position of 1VB1 is not needed to be increased by half length of component
#plt.text(5.7277,-27, '1VB1', fontsize=9, rotation=90, color='r')
##plt.text(6.36585,-27, '1VB1', fontsize=9, rotation=90, color='r') # half length of 1VB1 was added to position (wrong)
#plt.text(7.76224,-27, 'Q4', fontsize=9, rotation=90, color='r')
#plt.text(8.53186,-27, 'Q5', fontsize=9, rotation=90, color='r')
#plt.text(9.2964,-27, 'Q6', fontsize=9, rotation=90, color='r')
#plt.text(10.98042,-27, 'KICKER', fontsize=9, rotation=90, color='r')
#plt.text(14.17574,-27, '1BVB2', fontsize=9, rotation=90, color='r')
##Septum Length in G4B 1.003046, Yi-Nong used 555
##Regarding Beamline 1U Geometry, position of Septum is not needed to be increased by half length of component
#plt.text(16.78178,-27, 'SEPTUM ', fontsize=9, rotation=90, color='r')
##plt.text(17.05928,-27, 'SEPTUM ', fontsize=9, rotation=90, color='r') #Septum Length = 0.555, half length of Septum was added to position (wrong)
##plt.text(17.283303,-27, 'SEPTUM ', fontsize=9, rotation=90, color='r') #Septum Length = 1.003046, half length of Septum was added to position (wrong)
##1UB0 Length in G4B 1.601169, Yi-Nong used 1.200
##Regarding Beamline 1U Geometry, position of 1UB0 is not needed to be increased by half length of component
#plt.text(22.40534,-27, '1UB0', fontsize=9, rotation=90, color='r')
##plt.text(23.00534,-27, '1UB0', fontsize=9, rotation=90, color='r') #1UB0 Length 1.200, half length of 1UB0 was added to position (wrong)
##plt.text(23.2059245,-27, '1UB0', fontsize=9, rotation=90, color='r') #1UB0 Length 1.601169, half length of 1UB0 was added to position (wrong)
#plt.text(27.49296,-27, '1UQ1', fontsize=9, rotation=90, color='r')
#plt.text(27.91206,-27, '1UQ2', fontsize=9, rotation=90, color='r')
#plt.text(32.34182,-27, 'TARGET', fontsize=9, rotation=90, color='r')

#marking componet geometries
#Q1
    #plt.axvline(0.52952, ymax=0.6, linewidth=0.5, color = 'k')
plt.axvline(0.73147, ymax=0.535, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(0.73147, ymax=0.51, ymin=0.5, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(0.73147, ymax=0.32, ymin=0.22, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(0.93342, ymax=0.6, linewidth=0.5, color = 'k')
#Q2
    #plt.axvline(1.2747, ymax=0.63, linewidth=0.5, color = 'k')
plt.axvline(1.47915, ymax=0.515, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(1.47915, ymax=0.49, ymin=0.48, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(1.47915, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(1.6836, ymax=0.63, linewidth=0.5, color = 'k')
#Q3
    #plt.axvline(2.5345, ymax=0.6, linewidth=0.5, color = 'k')
plt.axvline(2.78, ymax=0.55, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(2.79995, ymax=0.52, ymin=0.50, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(2.79995, ymax=0.38, ymin=0.33, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(3.0654, ymax=0.6, linewidth=0.5, color = 'k')
#1VB1
    #plt.axvline(4.3950, ymax=0.68, linewidth=0.5, color = 'k')
    #plt.axvline(5.03315, ymax=0.54, ymin=0.51, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(5.03315, ymax=0.43, ymin=0.40, linewidth=11, color = 'grey', alpha=0.3)
plt.axvline(5.03315, ymax=0.57, linewidth=30, color = 'grey', alpha=0.3)
    #plt.axvline(5.6713, ymax=0.68, linewidth=0.5, color = 'k')
#Q4
    #plt.axvline(6.7994, ymax=0.74, linewidth=0.5, color = 'k')
plt.axvline(7.0613, ymax=0.675, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(7.0613, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(7.3232, ymax=0.74, linewidth=0.5, color = 'k')
#Q5
    #plt.axvline(7.5648, ymax=0.77, linewidth=0.5, color = 'k')
plt.axvline(7.83085, ymax=0.66, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(7.83085, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(8.0969, ymax=0.77, linewidth=0.5, color = 'k')
#Q6
    #plt.axvline(8.3897, ymax=0.74, linewidth=0.5, color = 'k')
plt.axvline(8.5929, ymax=0.69, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(8.5929, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(8.7961, ymax=0.74, linewidth=0.5, color = 'k')
#KICKER
    #plt.axvline(9.5258, ymax=0.77, linewidth=0.5, color = 'k')
plt.axvline(10.2759, ymax=0.63, linewidth=38, color = 'grey', alpha=0.3)
    #plt.axvline(10.2759, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(11.026, ymax=0.77, linewidth=0.5, color = 'k')
#SEPTUM
    #plt.axvline(15.800, ymax=0.75, linewidth=0.5, color = 'k')
plt.axvline(16.0775, ymax=0.57, linewidth=13, color = 'grey', alpha=0.3)
    #plt.axvline(16.0775, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(16.355, ymax=0.75, linewidth=0.5, color = 'k')
#CBY0
    #plt.axvline(19.277, ymax=0.75, linewidth=0.5, color = 'k')
#plt.axvline(19.502, ymax=0.52, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(19.502, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(19.727, ymax=0.75, linewidth=0.5, color = 'k')
#1UB0
    #plt.axvline(21.100, ymax=0.75, linewidth=0.5, color = 'k')
plt.axvline(21.7, ymax=0.52, linewidth=29, color = 'grey', alpha=0.3)
    #plt.axvline(20.5, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(22.300, ymax=0.75, linewidth=0.5, color = 'k')
#1UQ1 and 1UQ2, they are to close together to plot them seperately
    #plt.axvline(26.657, ymax=0.75, linewidth=0.5, color = 'k')
plt.axvline(26.9975, ymax=0.645, linewidth=15, color = 'grey', alpha=0.3)
    #plt.axvline(26.9975, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(27.338, ymax=0.75, linewidth=0.5, color = 'k')
#COLL
    #plt.axvline(29.612, ymax=0.89, linewidth=0.5, color = 'k')
plt.axvline(29.797, ymax=0.855,linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(29.797, ymax=0.22, ymin=0.16, linewidth=11, color = 'grey', alpha=0.3)
    #plt.axvline(29.982, ymax=0.89, linewidth=0.5, color = 'k')
#BeamlineEndWindow
#plt.axvline(31.1,  linewidth=3, color = 'grey', alpha=0.3)
#Target
    #plt.axvline(31.635, linewidth=1.3, color = 'k')
plt.axvline(31.96,  linewidth=14, color = 'grey', alpha=0.3)


#positions of all used virtual detectors
plt.axvline(0.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(0.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(1.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(1.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(2.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(2.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(3.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(3.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(4.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(4.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(5.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(5.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(5.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(6.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(7.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(7.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(8.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(8.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(8.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(8.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(8.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(9.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(9.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(9.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(9.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(11.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(11.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(11.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(12.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(12.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(13.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(13.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(14, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(14.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(14.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(15.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(15.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(15.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(16.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(16.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(16.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(16.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(16.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(16.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(17.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(18.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(19.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(20.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(20.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(20.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(20.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(20.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(21.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(22.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(22.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(22.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(22.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(22.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(22.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(23.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(24.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(25.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(26.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(26.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(26.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(26.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(26.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(26.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(26.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(27.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(27.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(27.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(27.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(27.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(27.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(27.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.8, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(28.9, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(29.0, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(29.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(29.2, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(29.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(29.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(29.5, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(29.6, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(30.1, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(30.4, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(30.7, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(31, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(31.3, ymax=0.03, linewidth=1.5, color = 'm')
plt.axvline(31.5, ymax=0.03, linewidth=1.5, color = 'm')


plt.text(0,-32, 'Beamline components', fontsize=15, color='b')
plt.text(0,-35, 'Virtual detectors', fontsize=15, color='m')
#plt.text(0,-35, 'Components regarding G4Beamline-code (for comparison)', fontsize=9, color='r')
#plt.title('Beam envelope BL1V/BL1U', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('distance [m]', fontsize=15) 
plt.ylabel('2\u03C3 beam size [mm]', fontsize=15) 
plt.savefig("C:\\Users\\Stefan\\Documents\\Masterarbeit\\G4Beamline\\BL1U\\BeamDevelopment\\BeamProfileDevelopment2.pdf", bbox_inches = "tight")
plt.show()

In [ ]:
#Determine beamsizes at the virtual detectors
#part1, execute all programms of all virtual detectors to fill results file with beamsizes (split into two lines, else kernel will die)
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\HARP0.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_10.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_20.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_30.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_40.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_50.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDQ1.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_100.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_110.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_120.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDQ2.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_170.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_180.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_190.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_200.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_210.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_220.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_230.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_240.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_250.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDQ3.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_310.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_320.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_330.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_340.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_350.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_360.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_370.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_380.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_390.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_400.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_410.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_420.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_430.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDVB1.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_570.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_580.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_590.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_600.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_610.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_620.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_630.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_640.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_650.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_660.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_670.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDQ4.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_740.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_750.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDQ5.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_810.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_820.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_830.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDQ6.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_880.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_890.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_900.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_910.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_920.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_930.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_940.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_950.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDKICK.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1110.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1120.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1130.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1140.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1150.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1160.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1170.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1180.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1190.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1200.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1210.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1220.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1230.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1240.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1250.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1260.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1270.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1280.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1290.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1300.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1310.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1320.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1330.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1340.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDBVB2.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1450.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1460.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1470.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1480.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1490.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1500.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1510.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1520.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1530.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1540.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1550.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1560.ipynb"))).run_notebook()

In [ ]:
#part2, execute all programms of all virtual detectors to fill results file with beamsizes (split into two lines, else kernel will die)
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1570.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDSEPT.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1640.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1650.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1660.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1670.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1680.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1690.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1700.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1710.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1720.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1730.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1740.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1750.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1760.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1770.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1780.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1790.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1800.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1810.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1820.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1830.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1840.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1850.ipynb"))).run_notebook() 
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1860.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1870.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1880.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1890.ipynb"))).run_notebook() 
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1900.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1910.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1920.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1930.ipynb"))).run_notebook() 
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1940.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1950.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1960.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1970.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1980.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_1990.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\HARP1.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2000.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2010.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2020.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2030.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2040.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2050.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2060.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2070.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2080.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2090.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2100.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDUB0.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2240.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2250.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2260.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2270.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2280.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2290.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2300.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2310.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2320.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2330.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2340.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2350.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2360.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2370.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2380.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2390.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2400.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2410.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2420.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2430.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2440.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2450.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2460.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2470.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2480.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2490.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2500.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2510.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2520.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2530.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2540.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2550.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2560.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2570.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2580.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2590.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2600.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2610.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2620.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2630.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2640.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2650.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2660.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDUQ1.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2700.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDUQ2.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2740.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2750.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2760.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2770.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2780.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2790.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2800.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2810.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2820.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\HARP2.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2830.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2840.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2850.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2860.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2870.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2880.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2890.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2900.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2910.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2920.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2930.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2940.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2950.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_2960.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VDCOL2.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3000.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3010.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3020.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3030.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3040.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3050.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3060.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3070.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3080.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3090.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3100.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\HARP3.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3110.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3120.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3130.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3140.ipynb"))).run_notebook()
NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3150.ipynb"))).run_notebook()
#NotebookRunner(read(open(r"C:\Users\Stefan\Documents\Masterarbeit\Jupyter\Beamsize\VD_3160.ipynb"))).run_notebook()